In [1]:
import matplotlib.pyplot as plt
import pandas as pd

from ipywidgets import interact, Dropdown

In [2]:
xls = pd.ExcelFile('inputs/layout.xlsx')

In [3]:
subclasses = pd.read_excel(xls, 'subclasse').set_index('Código').to_dict()['Descrição']

In [4]:
df = pd.read_csv('outputs/data.csv', sep = ';', encoding = 'utf-8')

In [5]:
subclass_codes = df['subclasse'].unique()

In [6]:
dict_subclasses = {}

In [7]:
for code in subclass_codes:
    dict_subclasses[code] = subclasses[code]

In [8]:
dict_subclasses = {key: value for key, value in sorted(dict_subclasses.items(), key = lambda item: item[1])}

In [9]:
def subclass_graph(subclass, df_balance):
    fig, ax = plt.subplots(figsize = (15, 10))

    plt.suptitle(f'Saldo de Empregos', fontsize = 20, fontweight = 'bold')
    plt.title(subclass, fontsize = 18, y = 1.035)

    graph = ax.bar(df_balance['competencia'].values, df_balance['saldo'].values, zorder = 2)

    for bar in graph:
        height = bar.get_height()
        y_pos = height * 1.035
        
        if height >= 0:
            v_align = 'bottom'
        else:
            v_align = 'top'

        ax.text(bar.get_x() + bar.get_width() / 2, y_pos, height, ha = 'center', va = v_align, fontweight = 'bold')

    plt.grid(axis = 'y', zorder = 0)
    plt.show()

In [10]:
def get_balance(subclass):
    df_balance = df.loc[df['subclasse'] == subclass].groupby('competencia').agg({'saldo': 'sum'}).reset_index()
    df_balance['competencia'] = df_balance['competencia'].astype(str)
    df_balance['competencia'] = df_balance['competencia'].apply(lambda x: x[4:] + '/' + x[2:4])
    
    return df_balance

In [11]:
dd_subclass = Dropdown(options = dict_subclasses.values())

In [12]:
@interact(subclass = dd_subclass)
def picker(subclass):
    subclass_code = list(dict_subclasses.keys())[list(dict_subclasses.values()).index(subclass)]
    
    balance = get_balance(subclass_code)
    subclass_graph(subclass, balance)

interactive(children=(Dropdown(description='subclass', options=('Comercio Varejista de Artigos de Armarinho', …

---